# ***SQL Para Data Science - Prueba***.
### Nombre(s): Thomas Peet, Braulio Águila, Camilo Ramírez
### Generación: G47
### Profesores: Alfonso Tobar - Sebastián Ulloa
### Fecha: 14-11-2022

## `Instrucciones: Para poder correr el notebook es necesario renombrar el archivo .env_example a .env y modificar su contenido agregando los datos de conexión del usuario en postgres`.

# Importación de librerías

In [1]:
import os
import pandas as pd
import psycopg2
import psycopg2.extras as extras
import helpers
from dotenv import load_dotenv
import glob
import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from feature_engine.imputation import  MeanMedianImputer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn import set_config
set_config(display='diagram')

---
# Parte 1

## Creación de la Base de Datos

In [2]:
load_dotenv()
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_DBHOST')
dbname = os.getenv('POSTGRES_DBNAME')
drop_all = os.getenv('DROP_DATABASES')=='True'

if drop_all:
    conn = psycopg2.connect(user=user, host=host, port=5432, password=password, database=dbname)
    conn.set_session(autocommit=True)

    # Obtención de Cursor
    cursor = conn.cursor();
    
    # Eliminación de base de datos en caso de existir
    cursor.execute("DROP DATABASE IF EXISTS prueba;")

    # Creación de sentencia para la base de datos
    sqlCreateDatabase = "create database "+ dbname +";"

    # Creacion de la base de datos en PostgreSQL
    cursor.execute(sqlCreateDatabase);
    cursor.close()
    conn.close()

## Obtención y tratamiento de las columnas del CSV

In [5]:
df = pd.read_csv('train_cupid.csv')
df.rename(columns = {'hispanic / latin':'hispanic_latin'}, inplace = True)
df.columns = df.columns.str.replace(" ", "_")

cols = df.columns.to_list()
for index, col in enumerate(cols):
    cols[index] +=' numeric'
    
cols = ', '.join(cols)

## Creación de la tabla

In [6]:
conn = psycopg2.connect(dbname=dbname, user=user, host=host, port=5432, password=password)

cursor = conn.cursor()

#Eliminando las tablas train_cupid y test_cupid si existen.
cursor.execute("DROP TABLE IF EXISTS train_cupid;")
cursor.execute("DROP TABLE IF EXISTS test_cupid;")

# Obteniendo un Cursor para las tablas
name_table_train = f"train_cupid ({cols})"
name_table_test = f"test_cupid ({cols})"

# Creación de las sentecias para las tablas
sqlTable_train = "create table "+name_table_train+';'
sqlTable_test = "create table "+name_table_test+';'

# Creando las tablas en PostgreSQL
cursor.execute(sqlTable_train)
cursor.execute(sqlTable_test)
conn.commit()

## Importación de los datos del CSV a las Tablas creadas

In [7]:
with open('train_cupid.csv', 'r') as f:    
    next(f) # Saltar la fila de los encabezados del CSV.
    cursor.copy_from(f, 'train_cupid', sep=',')

with open('test_cupid.csv', 'r') as f:    
    next(f) # Saltar la fila de los encabezados del CSV.
    cursor.copy_from(f, 'test_cupid', sep=',')

conn.commit()

## Añadiendo columna indice a las tablas train_cupid y test_cupid

In [8]:
cursor.execute('ALTER TABLE train_cupid ADD indice SERIAL PRIMARY KEY;')
cursor.execute('ALTER TABLE test_cupid ADD indice SERIAL PRIMARY KEY;')
conn.commit()
cursor.close()

---
# Parte 2

## Importando datos de train_cupid con Pandas

In [9]:
df = pd.read_sql('SELECT * FROM train_cupid', conn, index_col='indice')
df.info()

C:\Users\tomas\AppData\Local\Temp\ipykernel_5332\4210961192.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM train_cupid', conn, index_col='indice')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20081 entries, 1 to 20081
Data columns (total 98 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             20081 non-null  float64
 1   height                          20081 non-null  float64
 2   virgo                           20081 non-null  float64
 3   taurus                          20081 non-null  float64
 4   scorpio                         20081 non-null  float64
 5   pisces                          20081 non-null  float64
 6   libra                           20081 non-null  float64
 7   leo                             20081 non-null  float64
 8   gemini                          20081 non-null  float64
 9   aries                           20081 non-null  float64
 10  aquarius                        20081 non-null  float64
 11  cancer                          20081 non-null  float64
 12  sagittarius                     

## Segmentación de la data para los 3 vectores objetivos: [**single**, **seeing_someone**, **available**]

In [10]:
targets = ['single', 'seeing_someone', 'available']
data = {}
for target in targets:
    X = df.drop(columns=[target])
    y = df[target]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2, random_state=42)
    data[target] = {
        'X': X,
        'y': y,
        'X_train': X_train,
        'X_valid': X_valid,
        'y_train': y_train,
        'y_valid': y_valid,
    }

## Definición de los clasificadores a utilizar

In [11]:
modelos = {
    'm1' : GradientBoostingClassifier(random_state=42),
    'm2' : AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=1, n_estimators=5), random_state=42, n_estimators=100, learning_rate=1),
    'm3' : RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10),
    'm4' : SVC(random_state=42, probability=True),
    'm5' : DecisionTreeClassifier(random_state=42),
    'm6' : LogisticRegression(random_state=42, C=0.01),
    'm7' : BernoulliNB()
}

## Preprocesamiento

In [12]:
prep = Pipeline(steps=[
    ('num_imp', MeanMedianImputer(imputation_method='mean')),
    ('ord', OrdinalEncoder(encoding_method='ordered', variables='age', ignore_format=True)),
    ('sc', SklearnTransformerWrapper(StandardScaler(), variables=['age', 'height']))
])

## Fit y serialización de los modelos

In [13]:
for target in targets:
    print(target)
    for modelo in modelos.values():
        model_f = {'prep':prep, 'classifier':modelo}
        helpers.report_performance(
            # Esta función genera un pipeline con el subpipeline de preprocess y el modelo a entrenar.
            helpers.pipeline_maker(**model_f),  
            str(modelo.__class__).replace("'>", '').split('.')[-1],
            target,
            data[target]['X_train'],
            data[target]['X_valid'],
            data[target]['y_train'],
            data[target]['y_valid']
        )

single
GradientBoostingClassifier
              precision    recall  f1-score   support

         0.0       1.00      0.52      0.68       299
         1.0       0.96      1.00      0.98      3718

    accuracy                           0.96      4017
   macro avg       0.98      0.76      0.83      4017
weighted avg       0.97      0.96      0.96      4017

AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.99      0.52      0.68       299
         1.0       0.96      1.00      0.98      3718

    accuracy                           0.96      4017
   macro avg       0.98      0.76      0.83      4017
weighted avg       0.97      0.96      0.96      4017

RandomForestClassifier
              precision    recall  f1-score   support

         0.0       1.00      0.51      0.68       299
         1.0       0.96      1.00      0.98      3718

    accuracy                           0.96      4017
   macro avg       0.98      0.76      0.83      401

---
# Parte 3

## Importando datos de test_cupid con Pandas

In [14]:
df_test = pd.read_sql('SELECT * FROM test_cupid', conn, index_col='indice')
df_test

C:\Users\tomas\AppData\Local\Temp\ipykernel_5332\4109272332.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql('SELECT * FROM test_cupid', conn, index_col='indice')


,age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,...,orientation_straight,sex_m,smokes_sometimes,smokes_trying_to_quit,smokes_when_drinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university
indice,,,,,,,,,,,,,,,,,,,,,
1,22.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,32.0,65.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,24.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,29.0,62.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5,39.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19939,48.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19940,52.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19941,59.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Función para insertar datos en una tabla

In [15]:
def execute_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    
    # SQL query que se ejecutará
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    print(f'\t\t{query}')
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("\t\tthe dataframe is inserted")
    cursor.close()

## Queries que se deben evaluar

In [16]:
queries = {
    'Query 1' : ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'],
    'Query 2' : ['income_over_75', 'french', 'german', 'orientation_straight', 'new_york'],
    'Query 3' : ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed'],
    'Query 4' : ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']
}

## Evaluación de los modelos y guardado de las tablas tabulación cruzada de las predicciones en la base de datos.

In [17]:
cursor_test = conn.cursor()
for target in targets:
    print(target)

    # Segmentación de la data para el test
    X_test = df_test.drop(columns=[target])
    y_test = df_test[target]
    
    # Fecha de los modelos que se utilzaran. Formato: 1211-19 = 12 de Noviembre a las 19 horas. Se utilizará por defecto la fecha actual
    fecha = datetime.datetime.now().strftime('%d%m-%H')
    
    # Listado de los modelos serializados
    archivos_de_modelos = glob.glob(f'./models/{target}*_{fecha}.pkl')
    
    # Se evaluaran las 4 queries en los 7 modelos entrenados para predecir las 3 variables objetivos. Es decir se evaluaran 84 modelos.
    for key, query in queries.items():
        print(f'\t{key}({query})')
        for archivo_modelo in archivos_de_modelos:
            print(f'\t\tModelo: {archivo_modelo}')

            # Se hace una predicción sobre la data de test y se crea una tabla de tabulación cruzada sobre la data usando las variables de la query
            queries_result_for_this_model, target_name, model_name = helpers.create_crosstab(archivo_modelo, X_test, y_test, query)
            
            # Se genera una tabla usando los datos del vector objetivo, nombre del modelo y query utilizada
            nombre_tabla = f"{target}_{model_name.lower()}_{key.lower().replace(' ','')}"

            # Las columnas que llevará esta tabla son las de la query más una variable y_hat que tendrá el promedio de la predicción para cada combinación
            cols = ' numeric ,'.join(list(queries_result_for_this_model.index.names))+' numeric'
            
            # Se elimina si existe y se crea la tabla
            cursor_test.execute(f"DROP TABLE IF EXISTS {nombre_tabla};")
            cursor_test.execute(f"CREATE TABLE {nombre_tabla} ({cols}, {target}_yhat numeric);")
            conn.commit()
            
            # Se insertan los valores de la tabulación cruzada en la tabla
            execute_values(conn, queries_result_for_this_model.reset_index(), nombre_tabla)

cursor_test.close()
conn.close()

single
	Query 1(['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'])
		Modelo: ./models\single__AdaBoostClassifier__1311-20.pkl
		INSERT INTO single_adaboostclassifier_query1(atheism,asian,employed,pro_dogs,chinese,single_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\single__BernoulliNB__1311-20.pkl
		INSERT INTO single_bernoullinb_query1(atheism,asian,employed,pro_dogs,chinese,single_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\single__DecisionTreeClassifier__1311-20.pkl
		INSERT INTO single_decisiontreeclassifier_query1(atheism,asian,employed,pro_dogs,chinese,single_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\single__GradientBoostingClassifier__1311-20.pkl
		INSERT INTO single_gradientboostingclassifier_query1(atheism,asian,employed,pro_dogs,chinese,single_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\single__LogisticRegression__1311-20.pkl
		INSERT INTO single_logisticregression_query1(atheism,asian,employed,